In [19]:
import pandas as pd

train_data = pd.read_csv('data/QA_train_data.csv')
test_data = pd.read_csv('data/QA_test_data.csv')

train_data.head()

,Question,Answer
0,From which country is the film A Beautiful Mind?,America
1,When was the film A beautiful Mind released?,2001
2,Of which genre is the film A Beautiful Mind?,biographical drama
3,What is John Nash's profession?,mathematician
4,Which prices did John Nash win?,Nobel price in Economics and Abel Prize


In [20]:
# pip install spacy
import spacy

# python -m spacy download en_core_web_md
nlp = spacy.load('en_core_web_md')

def preprocess(text: str) -> str:
    """
    Preprocesses the input text by lowercasing, removing punctuation, and lemmatizing.
    :param text: string
    :return: preprocessed string
    """
    doc = nlp(text.lower())
    tokens = [token.lemma_ for token in doc if not token.is_punct]
    return ' '.join(tokens)

# Preprocess the training questions
train_data['Processed_Question'] = train_data['Question'].apply(preprocess)


def predict_answer(question: str) -> str:
    """
    Predicts an answer to a given question
    :param question: string
    :return: answer string or None if insufficient confidence
    """
    q = nlp(preprocess(question))
    best_match = None
    best_similarity = 0.0

    for question, answer in zip(train_data['Processed_Question'], train_data['Answer']):
        similarity = q.similarity(nlp(question))
        if similarity > best_similarity:
            best_similarity = similarity
            best_match = answer

    if best_similarity > 0.95:  # Confidence threshold
        return best_match
    return None


# Predict
predicted = test_data['Question'].apply(lambda q: predict_answer(q))
actual = test_data['Answer']

In [21]:
# Evaluate

def evaluate_result(predicted: str, actual: str) -> int:
    """
    Evaluates individual string answer string pair
    :param predicted: answer predicted by predictor
    :param actual: correct answer according to data set
    :return: +1 if predicted answer is correct, 0 if no answer is predicted (None), -1 if answer is wrong
    """
    if predicted is None:
        return 0
    elif predicted == actual:
        return 1
    else:
        return -1

def evaluate_results(predicted: pd.Series, actual: pd.Series) -> float:
    """
    Evaluates a series of answer pairs
    :param predicted: Series of predicted answers
    :param actual: Series of correct answers
    :return: Float value between -1 (worst) and +1 (best)
    """
    sum = 0
    for index, value in predicted.items():
        eval = evaluate_result(value, actual[index])
        sum += eval
    result = sum / predicted.size
    return result


score = evaluate_results(predicted, actual)
print(f"Score: {score}")

Score: 0.30303030303030304
